# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [44]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.030147258660853
 1.00450425361187
 1.854192902627506

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [45]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0         0.0       0.0
 0.0587893   1.0       0.0
 0.228667   -0.331678  1.0
U factor:
3×3 Matrix{Float64}:
 0.717002  0.89336   0.243831
 0.0       0.355676  0.565465
 0.0       0.0       0.886033

In [46]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [47]:
Alu.P

3×3 Matrix{Float64}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [48]:
Alu.L

3×3 Matrix{Float64}:
 1.0         0.0       0.0
 0.0587893   1.0       0.0
 0.228667   -0.331678  1.0

In [49]:
Alu.U

3×3 Matrix{Float64}:
 0.717002  0.89336   0.243831
 0.0       0.355676  0.565465
 0.0       0.0       0.886033

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [50]:
A\b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999999
 1.0000000000000002

In [51]:
Alu\b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999999
 1.0000000000000002

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [52]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [53]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 -0.736716  -0.912019  -0.438335
  0.0        0.374683   0.31982
  0.0        0.0       -0.818579

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [54]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

In [55]:
Aqr.R

3×3 Matrix{Float64}:
 -0.736716  -0.912019  -0.438335
  0.0        0.374683   0.31982
  0.0        0.0       -0.818579

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [56]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -1.5116801489481522
 -0.435740076646876
  2.6920113294007804
vectors:
3×3 Matrix{Float64}:
 -0.379947   0.799821  -0.46468
 -0.577654  -0.597515  -0.55614
  0.722466  -0.05712   -0.689043

The values and the vectors can be extracted from the Eigen type by special indexing

In [57]:
AsymEig.values

3-element Vector{Float64}:
 -1.5116801489481522
 -0.435740076646876
  2.6920113294007804

In [58]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.379947   0.799821  -0.46468
 -0.577654  -0.597515  -0.55614
  0.722466  -0.05712   -0.689043

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [59]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           7.77156e-16  -1.58207e-15
 -1.66533e-16   1.0          -2.38698e-15
 -1.52656e-15  -1.77636e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [60]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [61]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [62]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.4519404653356618

In [63]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [64]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [65]:
@time eigvals(Asym);

  0.117572 seconds (11 allocations: 7.989 MiB)


In [66]:
@time eigvals(Asym_noisy);

  0.407669 seconds (14 allocations: 7.928 MiB)


In [67]:
@time eigvals(Asym_explicit);

  0.118621 seconds (11 allocations: 7.989 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [68]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.381856 seconds (17 allocations: 183.106 MiB, 4.48% gc time)


6.43212305672235

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [69]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [70]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//2   9//10  4//5
 1//10  2//5   3//5
  1     7//10  3//10

In [71]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 11//5
 11//10
   2

In [72]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1
 1
 1

In [73]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
  1      0    0
 1//2    1    0
 1//10  3//5  1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1   7//10   3//10
 0  11//20  13//20
 0    0      9//50

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [81]:
using LinearAlgebra
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
A_eigv = eigen(A).values

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514488
  542.467730146614

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514488
  542.467730146614

In [82]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

AssertionError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [83]:
A_diag = diagm(A_eigv)

5×5 Matrix{Float64}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [89]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

AssertionError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [87]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [88]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]